imports

In [1]:
from HGNN.train.configParser import ConfigParser
from HGNN.train import dataLoader, CNN
from myhelpers import config_plots, TrialStatistics
config_plots.global_settings()

import os
import torch
from tqdm.notebook import tqdm as tqdm

params

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data/"
experimentName="BestModelForJeremy"
cuda=6
params = {"image_path": "INHS_cropped", 
                     "suffix": "52", 
                     "training_count": 0.64, 
                     "validation_count": 0.16} 

# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)


    
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
params = config_parser.fixExperimentParams(params)    
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
datasetManager.updateParams(config_parser.fixPaths(params))
dataset = datasetManager.getDataset()

# instantiate trial stat object
results_dir = os.path.join(experimentPathAndName, "results")
ts = TrialStatistics.TrialStatistics(results_dir)

using cuda 6
Creating dataset...


Loading images: 100%|██████████| 2600/2600 [02:34<00:00, 16.88it/s, fileName=/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588.jpg]  

Creating dataset... Done.


Get Random scores

In [3]:
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

fineList = dataset.csv_processor.getFineList()
numberOffine = len(fineList)

lbllist=torch.zeros(0)
predlist=torch.zeros(0)
with torch.no_grad():
    for batch in test_loader:
        classes = batch["fine"]
        if torch.cuda.is_available():
            classes = classes.cpu()
        lbllist=torch.cat([lbllist,classes.float().view(-1)])
        predlist=torch.cat([predlist, torch.FloatTensor(classes.size()[0]).random_(0, numberOffine)])
        
ts.addTrialPredictions(params, predlist, lbllist, range(numberOffine))
conf_mat = ts.printTrialConfusionMatrix(params, fineList, True)

from sklearn.metrics import f1_score
print("Macro f1:", f1_score(lbllist.cpu(), predlist.cpu(), average='macro'))

Transforming images:   0%|          | 8/2600 [00:00<00:37, 69.93it/s]

Loading saved indices...
file /home/elhamod/HGNN/experiments/BestModelForJeremy/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc/trainingIndex.csv read
file /home/elhamod/HGNN/experiments/BestModelForJeremy/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc/valIndex.csv read
file /home/elhamod/HGNN/experiments/BestModelForJeremy/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc/testIndex.csv read
Creating loaders...
Creating loaders... Done.


Transforming images: 100%|██████████| 2600/2600 [00:57<00:00, 45.51it/s]


Macro f1: 0.014321511821511822
